In [1]:
!python3 --version

Python 3.10.12


#### fix for could not import -lmza
https://github.com/lucidrains/imagen-pytorch/issues/92 

#### Fix for Can't import datasets AttributeError: partially initialized module 'datasets' has no attribute 'utils' (most likely due to a circular import)

Restart juypyter and kernel


In [2]:
import helper
import time
helper.check_gpu_capacity()

GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB


In [3]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")

# dataset size: 15011


/home/ubuntu/ai-project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1255.40it/s]
                                                        

Dataset json downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.
dataset size: 15011


In [4]:
sample_ds = dataset[randrange(len(dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
where do aliens come from
--> context

--> response
space the final frontier or is it?
--> category
open_qa


In [5]:
set(dataset['category'])

{'brainstorming',
 'classification',
 'closed_qa',
 'creative_writing',
 'general_qa',
 'information_extraction',
 'open_qa',
 'summarization'}

In [6]:
# len(dataset.map( ['context']))

In [7]:
# list(filter(lambda x :x != '', list(map(lambda x:((x['category'], len(x['context'])) if len(x['context']) > 5000 else ''), dataset))))

Based on our use case, we want to focus on 'general_qa' and  'information_extraction tasks.,

In [8]:
target_categories = ['closed_qa', 'summarization', 'information_extraction']
filtered_dataset = dataset.filter(lambda x: x['category'] in target_categories)

print(set(filtered_dataset['category']))
filtered_dataset

{'summarization', 'information_extraction', 'closed_qa'}


Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 4467
})

In [9]:
sample_ds = filtered_dataset[randrange(len(filtered_dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
In the following passage, replace Tallinn with Helsinki and Estonian with Finland.
--> context
Urmas was born in Tallinn. He graduated from the University of Tartu in 1996 with a BA in political science and continued his graduate studies there, but without obtaining the degree. He was active in journalism during his studies, first at Estonian Radio and later at Postimees, a mainstream daily and one of Estonia's most popular newspapers.
--> response
Urmas was born in Helsinki. He graduated from the University of Tartu in 1996 with a BA in political science and continued his graduate studies there, but without obtaining the degree. He was active in journalism during his studies, first at Finland Radio and later at Postimees, a mainstream daily and one of Finland's most popular newspapers.
--> category
information_extraction


To instruct tune our model, we need to convert our structured examples into a collection of tasks described via instructions. We define a formatting_function that takes a sample and returns a string with our format instruction.

In [10]:
def format_instruction(sample):
	return f"""### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.
            
            ### Input:
            {sample['response']}
            
            ### Response:
            {sample['instruction']}
        """


Let's test our formatting function on a random example.

In [11]:
from random import randrange

print(format_instruction(filtered_dataset[randrange(len(filtered_dataset))]))

## Do I need to add the context in the format_instruction?????

### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.
            
            ### Input:
            Hard magic is always strictly constrained by a set of rules which can simulate a more ordered world to the reader, while soft magic if more flexible, has no clearly defined limitations and can be used to imbue a sense of wonder to the reader.
            
            ### Response:
            According to Sanderson's Law of Magic, what is the difference between hard and soft magic?
        


In [12]:
print(format_instruction(sample_ds))

### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.
            
            ### Input:
            Urmas was born in Helsinki. He graduated from the University of Tartu in 1996 with a BA in political science and continued his graduate studies there, but without obtaining the degree. He was active in journalism during his studies, first at Finland Radio and later at Postimees, a mainstream daily and one of Finland's most popular newspapers.
            
            ### Response:
            In the following passage, replace Tallinn with Helsinki and Estonian with Finland.
        


### Instruction-tune Llama 2 using trl and the SFTTrainer

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False
# COMMENT IN TO USE FLASH ATTENTION
# replace attention with flash attention
# if torch.cuda.get_device_capability()[0] >= 8:
#     from utils.llama_patch import replace_attn_with_flash_attn
#     print("Using flash attention")
#     replace_attn_with_flash_attn()
#     use_flash_attention = True

helper.check_gpu_capacity()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB


In [15]:
# Hugging Face model id
model_llama_2_7b_id = "NousResearch/Llama-2-7b-hf" # non-gated
# model_id = "meta-llama/Llama-2-7b-hf" # gated

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model_llama_2_7b = AutoModelForCausalLM.from_pretrained(model_llama_2_7b_id, 
                                                        quantization_config=bnb_config, 
                                                        use_cache=False, 
                                                        device_map="auto")
tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.03s/it]


In [18]:
# from datasets import load_dataset
# from random import randrange


prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample_ds['response']}

### Response:
"""


input_ids = tokenizer_llama_2_7b(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model_llama_2_7b.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample_ds['response']}\n")
print(f"Generated instruction:\n{tokenizer_llama_2_7b.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample_ds['instruction']}")


Prompt:
Urmas was born in Helsinki. He graduated from the University of Tartu in 1996 with a BA in political science and continued his graduate studies there, but without obtaining the degree. He was active in journalism during his studies, first at Finland Radio and later at Postimees, a mainstream daily and one of Finland's most popular newspapers.

Generated instruction:
After a few months of a job as a writer for a business newspaper, he became an editor at the independent online news website Delfi.lt in 2002. He was promoted to the position of managing editor in 2005. He is known for his anti-Lithuanian government stance. After the Lithuanian government accused him of spreading disinformation, the government took legal action against the editor of Delfi, and in 2
Ground truth:
In the following passage, replace Tallinn with Helsinki and Estonian with Finland.


In [20]:
## From above, we see that the generated response is not okay

In [19]:

model_llama_2_7b.config.pretraining_tp = 1

# Validate that the model is using flash attention, by comparing doc strings
if use_flash_attention:
    from utils.llama_patch import forward
    assert model_llama_2_7b.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"


# tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)
tokenizer_llama_2_7b.pad_token = tokenizer_llama_2_7b.eos_token
tokenizer_llama_2_7b.padding_side = "right"

The SFTTrainer supports a native integration with peft, which makes it super easy to efficiently instruction tune LLMs. We only need to create our LoRAConfig and provide it to the trainer.

In [20]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

# prepare model for training
model_llama_2_7b = prepare_model_for_kbit_training(model_llama_2_7b)
model_llama_2_7b = get_peft_model(model_llama_2_7b, peft_config)


Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use.

In [21]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="llama-7-int4-dolly-filtered",
    num_train_epochs=3,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)


In [22]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model_llama_2_7b,
    train_dataset=filtered_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer_llama_2_7b,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


In [23]:
helper.check_gpu_capacity()
# del model_llama_2_7b
# del model_llama_2_7b_kbit
helper.free_gpu_memory()
helper.memory_stats()

GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 16835 MB
  Used Memory: 5680 MB
GPU Memory Used: 4.31 GB
GPU Memory Cached: 4.49 GB


In [24]:
%%time

# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.5245, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 1.2996, 'learning_rate': 0.0002, 'epoch': 0.04}
{'loss': 1.2252, 'learning_rate': 0.0002, 'epoch': 0.05}
{'loss': 1.1681, 'learning_rate': 0.0002, 'epoch': 0.07}
{'loss': 1.139, 'learning_rate': 0.0002, 'epoch': 1.02}
{'loss': 1.0975, 'learning_rate': 0.0002, 'epoch': 1.03}
{'loss': 1.1151, 'learning_rate': 0.0002, 'epoch': 1.05}
{'loss': 1.1048, 'learning_rate': 0.0002, 'epoch': 1.07}
{'loss': 1.0805, 'learning_rate': 0.0002, 'epoch': 2.02}
{'loss': 1.0686, 'learning_rate': 0.0002, 'epoch': 2.03}
{'loss': 1.0996, 'learning_rate': 0.0002, 'epoch': 2.05}
{'loss': 1.0902, 'learning_rate': 0.0002, 'epoch': 2.07}
{'train_runtime': 3191.6811, 'train_samples_per_second': 4.199, 'train_steps_per_second': 0.524, 'train_loss': 1.1720665496242932, 'epoch': 2.07}
CPU times: user 26min 56s, sys: 26min 20s, total: 53min 16s
Wall time: 53min 11s


## Test Model and run Inference

In [26]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()

import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

args.output_dir = "llama-7-int4-dolly-filtered"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.19s/it]


Let’s load the dataset again with a random sample to try to generate an instruction.

In [38]:
# from datasets import load_dataset
# from random import randrange


# # Load dataset from the hub and get a sample
# dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
# sample = dataset[randrange(len(dataset))]

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample_ds['response']}

### Response:
"""

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample_ds['response']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.7)

print(f"Prompt:\n{sample_ds['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample_ds['instruction']}")


Prompt:
Urmas was born in Helsinki. He graduated from the University of Tartu in 1996 with a BA in political science and continued his graduate studies there, but without obtaining the degree. He was active in journalism during his studies, first at Finland Radio and later at Postimees, a mainstream daily and one of Finland's most popular newspapers.

Generated instruction:
Where was Urmas Paet born?
        
Ground truth:
In the following passage, replace Tallinn with Helsinki and Estonian with Finland.
